In [1]:
import numpy as np
from ase import Atoms
from copy import deepcopy
from itertools import combinations
import pandas as pd
from ase.io import read, write
from configurations import build_lattice, create_configurations
import sys
sys.path.insert(0,'../pycp2k')
sys.path.insert(0,'../python-utils')
from graphene_n_ions import too_close

In [2]:
from ase.visualize import view

In [3]:
from ase.build import molecule

In [4]:
cn = molecule('CN')
cn.positions

array([[ 0.      ,  0.      , -0.611046],
       [ 0.      ,  0.      ,  0.523753]])

In [5]:
cn.positions
cn.set_cell([10,10,10])
cn.pbc = True
cn.center()
cn.rotate(v='x',a=np.pi/2., center='COM')

In [6]:
view(cn)

In [7]:
lattice = build_lattice(4, motif=cn,lattice_constant=3)

In [8]:
lattice

[Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...)]

In [9]:
conf = Atoms()

In [10]:
for point in lattice:
    conf.extend(point)

In [11]:
view(conf)

In [12]:
confs = create_configurations(lattice,axis = 'z', angle = 180.0)

In [13]:
confs

{'AAAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AAAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AACA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AACC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACCA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACCC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CAAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CAAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CACA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CACC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCCA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCCC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...)}

In [14]:
view(confs.values())

In [41]:
bmim_pf6_opt = read('./opt_coords_bigbox.xyz')
bmim_pf6_opt.cell = [44.20800018, 42.54000092, 50.]
bmim_pf6_opt.center()

In [42]:
view(bmim_pf6_opt)

In [43]:
pf6 = bmim_pf6_opt[0:7]; bmim = bmim_pf6_opt[7:32]; electrode = bmim_pf6_opt[32:]

In [44]:
pair = pf6+bmim
eq_dist_ion_pair_electrode = abs(electrode.get_center_of_mass()-pair.get_center_of_mass())[2]
eq_dist_ion_pair_electrode

4.4099961471344038

In [45]:
eq_dist_ion_pair = np.linalg.norm(bmim.get_center_of_mass() - pf6.get_center_of_mass())
eq_dist_ion_pair

5.0451744679708543

In [19]:
view(pair)

In [46]:
pair.rotate('y',a=np.pi/2,center='COM')
pair.rotate('x',a=np.pi/9.,center='COM')

In [21]:
view(pair)

In [47]:
electrode.center()
pair.center()
pair.translate([0.0,0.0,-eq_dist_ion_pair_electrode])

In [23]:
view(pair)

In [48]:
vector = 'z'
rotation_angle = 180.0
separation = eq_dist_ion_pair
pairs = 2
lshift = -5.7
rshift = 5.7
add_electrode = True

In [49]:
lattice = build_lattice(pairs, motif=pair,lattice_constant=separation)
l_confs = create_configurations(lattice,vector,rotation_angle)
######
for key in l_confs.keys():
    l_confs[key].set_cell(bmim_pf6_opt.cell)
    l_confs[key].center(axis=(0,1))
    if lshift != 0.0:
        l_confs[key].center(axis=2)
        l_confs[key].translate([0.0,0.0,lshift])
        if add_electrode and too_close(l_confs[key]+electrode,1.0):
            (l_confs[key]+electrode).write('lhs_'+key+'_positions.vasp')
            print('LHS ions are too close to the electrode')
            sys.exit()

lattice = build_lattice(pairs, motif=pair,lattice_constant=separation)
r_confs = create_configurations(lattice,vector,rotation_angle)
for key in r_confs.keys():
    r_confs[key].set_cell(bmim_pf6_opt.cell)
    r_confs[key].rotate('y',a=np.pi,center='COM')
    r_confs[key].center(axis=(0,1,2))
    r_confs[key].translate([0.0,0.0,eq_dist_ion_pair_electrode])
    if rshift != 0.0:
        r_confs[key].center(axis=2)
        r_confs[key].translate([0.0,0.0,rshift])
        if add_electrode and too_close(r_confs[key]+electrode,1.0):
            (r_confs[key]+electrode).write('rhs_'+key+'_positions.vasp')
            print('RHS ions are too close to the electrode')
            sys.exit()

In [50]:
full_confs = []
for  l_key in l_confs.keys():
    for  r_key in r_confs.keys():
        box = l_confs[l_key] + r_confs[r_key]
        if add_electrode:
            box.extend(electrode)
        box.write('L-'+l_key+'-R-'+r_key+'-positions.cif')
        full_confs.append(box)

In [51]:
view(full_confs)